# Build secrets in Docker Compose, the secure way

When you’re building a Docker image, you might need to use some secrets: the password to a private package repository, for example. You don’t want this secret to end up embedded in the image, because then anyone who somehow gets access to the image will get access to your private repository.

> Note: If you’re thinking “why not just use environment variables?”, those are used for runtime secrets once the image is built. This article is about build secrets, which are used while building the image via a Dockerfile.

Newer versions of Docker support secrets via the experimental BuildKit backend, and Docker Compose 1.25 and later can build images using BuildKit. Unfortunately, as of March 2020 the ability to pass in secrets securely from Compose is still stuck as a work in progress.

In this article I’ll demonstrate a technique that allows you use the same Dockerfile to both build production images securely with secrets, while still enabling easy development with Docker Compose.

## Two use cases for your Dockerfile

It’s handy to use the same Dockerfile both for production use, and for local development with Docker Compose. Typically you’d use your Dockerfile with Compose’s build functionality:

```yml
version: "3.7"
services:
  yourapp:
    build:
      context: "."
```

You can then do:

    docker-compose up

And it will (re)build the image for you, and then run it.

For production use, you build the image and then push it:

    docker build -t myimage .
    docker push myimage

So far, so good. But what if you need to build with a secret of some sort?

## A first, insecure attempt

Let’s say we have a script that needs a build secret, for example to download a Python package from a private DevPI repository. For simplicity, we’ll just have our use-secret.sh script print the secret, to prove we have it:

    #!/bin/bash
    set -euo pipefail

    echo "Secret is: $THEPASSWORD"

The naive way to pass in secrets is using Docker build args, since they’re supported everywhere, including Docker Compose.

```Dockerfile
FROM python:3.8-slim-buster
# Using ARG for build secrets is INSECURE!
ARG THEPASSWORD
COPY use-secret.sh .
RUN ./use-secret.sh
```

We can write a docker-compose.yml that passes in the secret:

```yml
version: "3.7"
services:
  yourapp:
    build:
      context: "."
      args:
        THEPASSWORD: "s3kr!t"
```

For local development we can run or build the image via Compose:

    docker-compose build | grep Secret

And that’s fine.

We can also build the image using Docker, as preparation for pushing it to an image registry:

    docker build -t myimage --build-arg THEPASSWORD=s3krit . | grep Secret

This is insecure: don’t ever do this. If we inspect the image layers, we’ll see that the secret has been embedded into the image!

    docker history myimage

Anyone who gets access to this image will know your password! What to do?

## BuildKit secrets, a partial solution

https://docs.docker.com/develop/develop-images/build_enhancements/

BuildKit is a new—and still experimental—implementation of Docker image building, which among other things adds support for securely using build secrets. And Docker Compose has experimental support for using BuildKit as of v1.25.

But there’s a problem: Docker Compose doesn’t yet support the secrets feature in BuildKit. There’s an in-progress PR, but as of March 2020 it hasn’t been merged, let alone released.

So what we’re going to do is combine the two approaches:
1. Docker Compose will continue to use build args to pass in secrets.
2. For the production image, built with docker build, we’ll use BuildKit to pass in secrets.

This will allow us to use the same Dockerfile for both local development and for our final production image.

## The combo approach

The way BuildKit secrets work is that a file with the secret gets mounted to a temporary location during the RUN command, e.g. /var/secrets/thepassword. Since it’s only mounted during a particular RUN command being run, it doesn’t end up embedded in the final image.



We modify use_secret.sh so that it checks if that file exists. If it does, it uses it to set the `$THEPASSWORD` environment variable; if the file doesn’t exist, we fall back to the environment variable. That means $THEPASSWORD can be set either by BuildKit secrets or via a build arg:

```shell
#!/bin/bash
set -euo pipefail
if [ -f /run/secrets/thepassword ]; then
   export THEPASSWORD=$(cat /run/secrets/thepassword)
fi

echo "Secret is: $THEPASSWORD"
```

Next, we modify the Dockerfile to enable BuildKit and mount the secret:

```Dockerfile
# syntax = docker/dockerfile:1.0-experimental
FROM python:3.8-slim-buster
# Only use the build arg for local development:
ARG THEPASSWORD
COPY use-secret.sh .
# Mount the secret to /run/secrets:
RUN --mount=type=secret,id=thepassword ./use-secret.sh
```

The docker-compose.yml file remains unchanged.

Now we need to set two environment variables, one to tell Docker to use BuildKit, one to tell Compose to use the CLI version of Docker and therefore BuildKit. We also write the secret to a file:

    $ export DOCKER_BUILDKIT=1
    $ export COMPOSE_DOCKER_CLI_BUILD=1
    $ echo 's3krit' > /tmp/mypassword

With Compose, the build arg is used:

    docker-compose build --progress=plain \
        --no-cache 2>&1 | grep Secret

Note that the --no-cache is just to ensure it actually rebuilds, in case you’ve run all the above yourself; in real usage you can omit it. The 2>&1 redirects stderr to stdout for easier grepping.

When we’re ready to build for production use, we use docker build with the BuildKit secrets feature:

    docker build --no-cache -t myimage \
        --secret id=thepassword,src=/tmp/mypassword \
        --progress=plain . 2>&1 | grep Secret

## Is it secure?

Let’s make sure our secret wasn’t leaked:



    docker history myimage

Success! We passed in the secret to the same Dockerfile using both Compose and docker build, and in the latter case we didn’t leak the build secret.